In [1]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
from functools import reduce
import xoak
import metpy.calc as mpcalc
from metpy.units import units

In [2]:
# Set up paths

path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
path_to_work = '/cw3e/mead/projects/cwp140/scratch/dnash/data/preprocessed/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
option = 'a'
### choose which temporal resolution for the precipitation data (hourly or daily)
temporal_res = 'hourly'

### variable name (PCPT, T2, UV)
varname = 'UV'

### TODO: make a yaml dict
ext1 = [-141., -130., 54., 61.] # extent of SEAK

lonmin, lonmax, latmin, latmax = ext1

## for josh and eliza
xs = [-132.770186, -132.983141, -132.380256]
ys = [55.217527, 55.546800, 55.545950]
lbl1 = ['Hydaburg', 'Klawock', 'Kasaan']

# ## For seak-clim
# xs = [-135.4519, -135.3277, -135.8894, -139.671, -133.1358, -132.4009]
# ys = [58.1122, 59.4538, 59.3988, 59.5121, 55.4769, 55.5400]
# lbl1 = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']


In [4]:
### TODO: make a yaml dict
ext1 = [-141., -130., 54., 61.] # extent of SEAK 

## six communities
# -135.4519	58.1122, Hoonah (PAOH) 
# -135.3277	59.4538, Skagway (PAGY)
# -135.8894, 59.3988, Klukwan
# -139.671	59.5121, Yakutat (PAYA)
# -133.1358, 55.4769, Craig
# -132.4009, 55.5400, Kasaan

# xs = [-135.4519, -135.3277, -135.8894, -139.671, -133.1358, -132.4009]
# ys = [58.1122, 59.4538, 59.3988, 59.5121, 55.4769, 55.5400]
# lbl1 = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
# lbl_align = ['center', 'left', 'right', 'center', 'right', 'center'] # where the labels go


### Import preprocessed SEAK-WRF precip data

In [5]:
fname_pattern = path_to_work + 'SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format(varname)
wrf = xr.open_mfdataset(fname_pattern, combine='by_coords')
        
if varname == 'UV':
    wrf = wrf.sel(lev='1000')
      
if (temporal_res == 'daily') & (varname == 'PCPT'):
    wrf = wrf.resample(time="1D").sum('time')

elif (temporal_res == 'daily') & (varname != 'PCPT'):
    wrf = wrf.resample(time="1D").mean('time')

elif (temporal_res == 'hourly'):
    wrf = wrf

In [6]:
wrf

<xarray.Dataset>
Dimensions:  (time: 350400, y: 250, x: 320)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2019-12-31T23:00:00
    lev      <U4 '1000'
    lat      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    U        (time, y, x) float32 dask.array<chunksize=(8760, 250, 320), meta=np.ndarray>
    V        (time, y, x) float32 dask.array<chunksize=(8760, 250, 320), meta=np.ndarray>
Attributes:
    contact:  rtladerjr@alaska.edu
    info:     Alaska CASC
    data:     Downscaled CFSR
    format:   version 1
    date:     Sat Dec  8 15:40:16 AKST 2018

## Generate precipitation time series for each community

In [9]:
%%time

df_lst2 = []
row_lbl2 = []
for i, (slon, slat) in enumerate(zip(xs, ys)):
    
    if option == 'a':
        ## select nearest grid cell to station
        points = xr.Dataset({"lat": slat, "lon": slon})
        wrf.xoak.set_index(["lat", "lon"], "sklearn_geo_balltree")
        ds = wrf.xoak.sel(lat=points.lat, lon=points.lon)
        
        if varname == 'UVDIR':
            ## calculate UV direction
            uvec = units.Quantity(ds['U'].values, "m/s")
            vvec = units.Quantity(ds['V'].values, "m/s")
            uvdir = mpcalc.wind_direction(uvec, vvec)
            ds = ds.assign(UV=lambda ds: uvdir)
            df = ds[varname].to_dataframe()
            df = df.rename(columns={varname: lbl1[i]}) # rename from varname to the name of the community
            df = df.reset_index() # remove the index
        
        elif varname == 'UV':
            ## build a dataframe
            d = {'uvec': ds['U'].values,
                 'vvec': ds['V'].values,
                 'time': ds.time.values}
            df = pd.DataFrame(d)
                 
            df = df.rename(columns={'uvec': '{0}_u'.format(lbl1[i]),
                                    'vvec': '{0}_v'.format(lbl1[i])})
            
        else:
            df = ds[varname].to_dataframe()
            df = df.rename(columns={varname: lbl1[i]}) # rename from varname to the name of the community
            df = df.reset_index() # remove the index
                 
    df_lst2.append(df)

CPU times: user 5.89 s, sys: 2min 6s, total: 2min 12s
Wall time: 36min 20s


In [12]:
## merge all dfs to one
df_merged = reduce(lambda x, y: pd.merge(x, y, on = 'time'), df_lst2)
## hack for weird behavior for daily df option a
if (option == 'a') & (varname == 'UVDIR'):
    df_merged = df_merged.drop(['lat_x', 'lat_y', 'lon_x', 'lon_y', 'lev_y', 'lev_x'], axis=1)
elif (option == 'a') & (varname == 'PCPT'):
    df_merged = df_merged.drop(['lat_x', 'lat_y', 'lon_x', 'lon_y', 'lat', 'lon'], axis=1)
    
df_merged

,Hydaburg_u,Hydaburg_v,time,Klawock_u,Klawock_v,Kasaan_u,Kasaan_v
0,-3.499726,-0.988729,1980-01-01 00:00:00,-3.968787,0.500454,-1.819762,-1.739944
1,-3.701401,-1.034299,1980-01-01 01:00:00,-4.098705,0.557126,-1.873655,-1.887690
2,-3.631013,-0.948625,1980-01-01 02:00:00,-3.882326,0.672850,-1.793747,-1.756102
3,-3.485517,-0.803394,1980-01-01 03:00:00,-3.606512,0.730619,-1.789532,-1.681777
4,-3.249341,-0.619768,1980-01-01 04:00:00,-3.316930,0.782472,-1.667291,-1.480793
...,...,...,...,...,...,...,...
350395,-2.552256,2.555224,2019-12-31 19:00:00,-1.677532,3.226216,-0.864751,2.877005
350396,-2.711598,2.223421,2019-12-31 20:00:00,-2.937131,2.703197,-2.729281,1.881494
350397,-3.100749,1.632389,2019-12-31 21:00:00,-2.431265,2.581412,-3.283737,1.222764
350398,-3.149729,1.781100,2019-12-31 22:00:00,-3.409823,2.322019,-3.641069,1.686587


In [13]:
## save to csv file
df_merged.to_csv(path_to_out + 'SEAK_{0}_{1}_{2}_josh-eliza.csv'.format(varname, option, temporal_res))